In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
from scipy import interpolate
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools
import math
from PyHEADTAIL.feedback.digital_processors import ADC,DAC,FIR_Filter,DigitalFilter
from PyHEADTAIL.feedback.processors import Lowpass, PhaseLinearizedLowpass, Sinc, NoiseGenerator

np.random.seed(0)

In [ ]:
class Signal(object):
    def __init__(self, z_bins,z,x,t):
        self.z_bins = z_bins
        self.z = z
        self.x = x
        self.t = t
        self.signal = np.array(x)
        
    def mean_x(self):
        return self.x
        
    def mean_z(self):
        return self.z

def impulse(f, n_points = 100, amplitude = 1.):
    t_bins = np.linspace(-1./f, 1./f, n_points + 1)

    t = np.array([(i + j) / 2. for i, j in zip(t_bins, t_bins[1:])])

    z_bins = c * t_bins
    z = c * t

    x = np.zeros(len(t))
    for i, val in enumerate(t):
        if val >= 0.:
            x[i] = amplitude
            break
    print np.amax(x)
            
    return Signal(z_bins, z, x, t)
    
def generate_signal(signal_generator, f, amplitude, n_periods, n_per_period, n_zero_periods):
    
    t_min = -1.*n_zero_periods[0]
    t_max = n_periods+n_zero_periods[1]
    t_bins = np.linspace(t_min,t_max,int((t_max-t_min)*n_per_period)+1)
    t = np.array([(i + j) / 2. for i, j in zip(t_bins, t_bins[1:])])

    x = np.zeros(len(t))
    signal_points = (t > 0.) * (t < n_periods)
    x[signal_points] = amplitude * signal_generator(t[signal_points])

    z_bins = c * t_bins / f
    z = c * t / f

    return Signal(z_bins, z, x, t / f )
    
    
def square_impulse(f,amplitude):
    def signal_generator(x):
        signal = np.zeros(len(x))
        for i, val in enumerate(x):

            if 0.< val % 1. < 0.5:
                signal[i] = 1.

            elif 0.5 < val % 1. < 1.0:
                signal[i] = -1.
        return signal

    return generate_signal(signal_generator, f, amplitude, 1, 100, (1,1))
    

def sine_impulse(f,amplitude):
    def signal_generator(x):
        return np.sin(2*pi*x)

    return generate_signal(signal_generator, f, amplitude, 1, 100, (1,1))

def sine_wave(f, amplitude = 1., n_periods=10., n_per_period = 100):
    def signal_generator(x):
        return np.sin(2*pi*x)
    n_zero_periods = (0, 0)
    return generate_signal(signal_generator, f, amplitude, n_periods, n_per_period, n_zero_periods)

def compare_signals(signals,labels,normalized = False, xlabel = None, ylabel = None, title = None):
    fig = plt.figure(figsize=(12, 7))
    ax1 = fig.add_subplot(111)
    
    ax2 = ax1.twiny()
    
    norm_coeff = 1.
    
    for i, (signal,label) in enumerate(zip(signals,labels)):
        if normalized and i == 0:
            norm_coeff = np.amax(signal.signal)

        ax1.plot(signal.t, signal.signal/norm_coeff,label=label)
        ax2.plot(signal.z, np.ones(len(signal.z))) # Create a dummy plot
        ax2.cla()

    
    if xlabel is not None:
        ax1.set_xlabel(xlabel)
    else:
        ax1.set_xlabel('Time [s]')
    
    if ylabel is not None:
        ax1.set_ylabel(ylabel)
    elif normalized:
        ax1.set_ylabel('Normalized amplitude')
    else:
        ax1.set_ylabel('Amplitude')
        
    ax1.legend(loc='upper right')
    
    ax2.set_xlabel('Distance [m]')
    if title is not None:
        title = ax1.set_title(title)
        title.set_y(1.1)
    
    plt.show()
    
def process_signal(signal,processors):
    
    for processor in processors:
        signal.signal = processor.process(signal.signal,signal,None)

In [ ]:
""" In this example the usage of the digital signal processors is demonstrated.

    @author Jani Komppula
    @date 26/09/2016
    @copyright CERN
"""

In [ ]:
f_s = 1e8 #[Hz] Frequency scale used in this example

In [ ]:
# Signal can be digitized by using an analog to digital converter (ADC) and a digital to analog 
# converter (DAC). Those processors can modify the signal depending on a sampling rate, a number of bits 
# and an input range of the DAC. This is demonstrated below.

sampling_rate = 2.5e9

signal_org = sine_impulse(f_s,1.0)

signal_1 = sine_impulse(f_s,1.0)
processors_1 = [ADC(sampling_rate),DAC(sampling_rate)]
process_signal(signal_1,processors_1)

signal_2 = sine_impulse(f_s,1.0)
processors_2 = [ADC(sampling_rate,8,(-20.,20.)),DAC(sampling_rate)]
process_signal(signal_2,processors_2)

signal_3 = sine_impulse(f_s,1.0)
processors_3 = [ADC(sampling_rate,8,input_range = (-0.5,0.5)),DAC(sampling_rate)]
process_signal(signal_3,processors_3)


compare_signals([signal_org, signal_1, signal_2, signal_3],
                ['Input signal', 'Sampling rate limited',  'n_bits limited',  'Range limited'])

In [ ]:
# Actual digital signal processors can be placed between an ADC and a DAC. In this example, 
# digital (FIR) filters are used. The processor namely FIR_filter calculater the coefficients with firwin(...)
# function of SciPy and to the processor namely DigitalFilter the coefficients can be set manually as an input
# parameter.

sampling_rate = 2.5e9

signal_org = sine_impulse(f_s,1.0)

signal_1 = sine_impulse(f_s,1.0)
processors_1 = [ADC(sampling_rate),
                DAC(sampling_rate)]
process_signal(signal_1,processors_1)


signal_2 = sine_impulse(f_s,1.0)
processors_2 = [ADC(sampling_rate),
                FIR_Filter(21,0.4e8, sampling_rate),
                DAC(sampling_rate)]
process_signal(signal_2,processors_2)


# Coefficients for the (not so) elegant bandpass filter 
coefs = [
#   -0.0028939000415581606,
#   0.004258269757704304,
#   0.005070495002707721,
#   0.006910079727955648,
#   0.00919955851032536,
#   0.011564872626788276,
#   0.01369461362138718,
#   0.015247787003055504,
#   0.01593303169907596,
#   0.015463835536988577,
#   0.01361314783173641,
#   0.01025563782069558,
#   0.005378713065879916,
  -0.0008828798213830393,
  -0.008258088144177387,
  -0.016333774133216995,
  -0.024574982075318663,
  -0.03236793417851592,
  -0.03905837054934558,
  -0.044012712254038355,
  -0.04666919480611765,
  -0.04659632091015434,
  -0.04354421188142093,
  -0.03747454203057326,
  -0.028582911124175155,
  -0.01729360299360126,
  -0.0042400547433324425,
  0.009776697726780973,
  0.02384867703724508,
  0.03702688145799186,
  0.04839894601447956,
  0.057160819120311436,
  0.06268453058260898,
  0.06457161202786266,
  0.06268453058260898,
  0.057160819120311436,
  0.04839894601447956,
  0.03702688145799186,
  0.02384867703724508,
  0.009776697726780973,
  -0.0042400547433324425,
  -0.01729360299360126,
  -0.028582911124175155,
  -0.03747454203057326,
  -0.04354421188142093,
  -0.04659632091015434,
  -0.04666919480611765,
  -0.044012712254038355,
  -0.03905837054934558,
  -0.03236793417851592,
  -0.024574982075318663,
  -0.016333774133216995,
  -0.008258088144177387,
  -0.0008828798213830393,
#   0.005378713065879916,
#   0.01025563782069558,
#   0.01361314783173641,
#   0.015463835536988577,
#   0.01593303169907596,
#   0.015247787003055504,
#   0.01369461362138718,
#   0.011564872626788276,
#   0.00919955851032536,
#   0.006910079727955648,
#   0.005070495002707721,
#   0.004258269757704304,
#   -0.0028939000415581606
]

signal_3 = sine_impulse(f_s,1.0)
processors_3 = [ADC(sampling_rate),DigitalFilter(coefs), DAC(sampling_rate)]
process_signal(signal_3,processors_3)


compare_signals([signal_org, signal_1, signal_2, signal_3],
                ['Input signal', 'Digitalized signal',  'SciPy firwin',  'Manual coefficients'])